<a href="https://colab.research.google.com/github/2j0123/WASSUP_EST_FINAL_Team4/blob/SW/Inception%20V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

# "utf-8" 대신 적절한 인코딩을 사용해야 함
with open("/content/train_anger.json", "r", encoding="cp949") as json_file:
    json_load = json.load(json_file)
json_load

[{'filename': '6oj439e3fbcc52759fb3093035b7c0ecc55c93543dae63ec1162f5f6bfc6e481l.jpg',
  'gender': '남',
  'age': 30,
  'isProf': '일반인',
  'faceExp_uploader': '분노',
  'bg_uploader': '교통/이동수단(엘리베이터 포함)',
  'annot_A': {'boxes': {'maxX': 2073.2546049999996,
    'maxY': 1486.204442,
    'minX': 1338.2201049999999,
    'minY': 539.5066419999999},
   'faceExp': '분노',
   'bg': '교통/이동수단(엘리베이터 포함)'},
  'annot_B': {'boxes': {'maxX': 2074.6636049999997,
    'maxY': 1490.044442,
    'minX': 1341.270105,
    'minY': 528.132642},
   'faceExp': '분노',
   'bg': '교통/이동수단(엘리베이터 포함)'},
  'annot_C': {'boxes': {'maxX': 2072.188605,
    'maxY': 1487.918442,
    'minX': 1340.0851049999999,
    'minY': 525.650642},
   'faceExp': '분노',
   'bg': '교통/이동수단(엘리베이터 포함)'}},
 {'filename': '65rsfe402042f34319e10128c1ab9614e2f967690a64a0016f2c49ff33fe7sy3z.jpg',
  'gender': '남',
  'age': 40,
  'isProf': '전문인',
  'faceExp_uploader': '분노',
  'bg_uploader': '문화재 및 유적지',
  'annot_A': {'boxes': {'maxX': 2889.526926369863,
    

In [ ]:
import json

# "utf-8" 대신 적절한 인코딩을 사용해야 함
with open("/content/val_anger.json", "r", encoding="cp949") as json_file:
    json_load = json.load(json_file)
json_load

[{'filename': '7uji1c8ab7778bc286c3cd90077847f9a906f00b210838ba57877a29abd60ffix.jpg',
  'gender': '남',
  'age': 40,
  'isProf': '전문인',
  'faceExp_uploader': '분노',
  'bg_uploader': '오락/공연시설',
  'annot_A': {'boxes': {'maxX': 2333.6550000000007,
    'maxY': 1321.8902999999998,
    'minX': 1635.0129999999997,
    'minY': 333.64478},
   'faceExp': '분노',
   'bg': '오락/공연시설'},
  'annot_B': {'boxes': {'maxX': 2333.46442581244,
    'maxY': 1321.8902999999996,
    'minX': 1596.7427665866094,
    'minY': 333.6447799999996},
   'faceExp': '분노',
   'bg': '오락/공연시설'},
  'annot_C': {'boxes': {'maxX': 2333.655,
    'maxY': 1321.8902999999998,
    'minX': 1635.013,
    'minY': 333.64478},
   'faceExp': '분노',
   'bg': '오락/공연시설'}},
 {'filename': 'cngjfbcef71af41ffc4c8fb1c08a461bc13bdb91bd6bc2cb537b002fb71fc6cew.jpg',
  'gender': '남',
  'age': 30,
  'isProf': '전문인',
  'faceExp_uploader': '분노',
  'bg_uploader': '교통/이동수단(엘리베이터 포함)',
  'annot_A': {'boxes': {'maxX': 2195.811009358465,
    'maxY': 1558.78560497

In [1]:
# Google Drive와 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 필요한 라이브러리 import
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# 데이터 경로 설정
base_dir = '/content/drive/MyDrive/감정따뜻쟁이/train'

# 이미지 데이터 생성기 설정
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# 이미지 데이터 로드
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

# Inception-v3 모델 불러오기 (사전 훈련된 가중치 포함)
base_model = InceptionV3(weights='imagenet', include_top=False)

# 분류기 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

# 새로운 모델 정의 (기존의 Inception-v3 모델과 분류기를 결합)
model = Model(inputs=base_model.input, outputs=predictions)

# 모든 레이어를 동결하여 기존의 가중치를 고정
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(train_generator, epochs=100)

# 새로운 이미지 분류 테스트
def classify_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(299, 299))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, 0)  # 이미지 배치 추가
    img_array /= 255.  # 이미지 정규화
    predictions = model.predict(img_array)
    emotion_labels = ['anger', 'happy', 'sadness', 'panic']
    predicted_label = emotion_labels[np.argmax(predictions)]
    return predicted_label

# 예시: 이미지 분류 테스트
test_image_path = '/content/drive/MyDrive/test_image.jpg'  # 테스트할 이미지 파일의 경로
predicted_label = classify_image(test_image_path)
print('Predicted emotion:', predicted_label)

Found 5369 images belonging to 4 classes.
Epoch 1/100
168/168 [==============================] - 422s 2s/step - loss: 1.5823 - accuracy: 0.2865
Epoch 2/100
168/168 [==============================] - 412s 2s/step - loss: 1.3344 - accuracy: 0.3567
Epoch 3/100
168/168 [==============================] - 410s 2s/step - loss: 1.2984 - accuracy: 0.3848
Epoch 4/100
168/168 [==============================] - 416s 2s/step - loss: 1.2721 - accuracy: 0.4023
Epoch 5/100
168/168 [==============================] - 412s 2s/step - loss: 1.2555 - accuracy: 0.4122
Epoch 6/100
168/168 [==============================] - 410s 2s/step - loss: 1.2367 - accuracy: 0.4390
Epoch 7/100
168/168 [==============================] - 406s 2s/step - loss: 1.2355 - accuracy: 0.4275
Epoch 8/100
168/168 [==============================] - 413s 2s/step - loss: 1.2172 - accuracy: 0.4515
Epoch 9/100
168/168 [==============================] - 406s 2s/step - loss: 1.2104 - accuracy: 0.4615
Epoch 10/100
168/168 [==================

In [3]:
# Google Colab에서 파일 업로드하기 위한 라이브러리 import
from google.colab import files

# 이미지 분류 함수 정의
def classify_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(299, 299))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, 0)  # 이미지 배치 추가
    img_array /= 255.  # 이미지 정규화
    predictions = model.predict(img_array)
    emotion_labels = ['anger', 'happy', 'sadness', 'panic']
    predicted_label = emotion_labels[np.argmax(predictions)]
    return predicted_label

# 사용자가 업로드한 파일을 받아서 이미지 분류하는 함수 정의
def classify_uploaded_images():
    uploaded = files.upload()  # 사용자로부터 파일 업로드 받기
    for filename in uploaded.keys():
        image_path = '/content/' + filename
        predicted_label = classify_image(image_path)
        print(f'{filename}의 감정은 {predicted_label}입니다.')

# 사용자가 원하는 만큼 사진 또는 영상을 업로드하고 분류
while True:
    classify_uploaded_images()
    continue_upload = input("더 업로드하시겠습니까? (yes/no): ")
    if continue_upload.lower() != 'yes':
        break

Saving 0pnnbde1bec084d09efe5011eacbd99aa3ef8ad004d0d15ae3381adc39bd9j0qo.jpg to 0pnnbde1bec084d09efe5011eacbd99aa3ef8ad004d0d15ae3381adc39bd9j0qo.jpg
1/1 [==============================] - 0s 29ms/step
0pnnbde1bec084d09efe5011eacbd99aa3ef8ad004d0d15ae3381adc39bd9j0qo.jpg의 감정은 happy입니다.
더 업로드하시겠습니까? (yes/no): yes


Saving okss9bb8ceb2bbef180fbee6d54ecefdd331d67f7fe3bb167c03ddeb5d890go7q.jpg to okss9bb8ceb2bbef180fbee6d54ecefdd331d67f7fe3bb167c03ddeb5d890go7q.jpg
1/1 [==============================] - 0s 27ms/step
okss9bb8ceb2bbef180fbee6d54ecefdd331d67f7fe3bb167c03ddeb5d890go7q.jpg의 감정은 happy입니다.
더 업로드하시겠습니까? (yes/no): no
